Run this notebook in the same virtual environment with superlinked server
to ensure the same version of `superlinked` framework.

In [1]:
!pip freeze | grep superlinked

superlinked==19.14.3
superlinked-server==1.5.0


In [2]:
from pathlib import Path
import sys

# depending on the user's setup
# we will try to find the superlinked_app directory
# and add it to the sys.path

cwd = Path.cwd()
if cwd.name == "superlinked-recipes":
    project_dir = cwd / "projects" / "hotel-search"
elif cwd.name == "notebooks":
    project_dir = cwd.parent
else:
    project_dir = cwd

superlinked_app_dir = project_dir / "superlinked_app"
assert superlinked_app_dir.exists(), (
    f"{superlinked_app_dir} does not exist\n"
    "are you sure you are in the hotel-search/notebooks directory?"
)

if str(project_dir) not in sys.path:
    sys.path.append(str(project_dir))
    print("project_dir is added to sys.path")
else:
    print("project_dir is already in sys.path")

project_dir is added to sys.path


In [3]:
from superlinked import framework as sl

from superlinked_app.index import index, hotel_schema
from superlinked_app.query import query
from superlinked_app.api import vector_database

import pandas as pd

In [4]:
source = sl.InteractiveSource(hotel_schema)
executor = sl.InteractiveExecutor(
    sources=[source],
    indices=[index],
    vector_database=vector_database,
)
app = executor.run()

In [5]:
params = {
    "natural_query": "well rated pet friendly hotel in london, cozy interior",
    "limit": 3,
}

result = app.query(query, **params)
print(result.metadata.search_params)
sl.PandasConverter.to_pandas(result)

{'price_weight': 0.0, 'rating_weight': 1.0, 'rating_count_weight': 0.0, 'description_weight': 1.0, 'description': 'Pet-friendly hotel with cozy interior.', 'limit': 3, 'select_param__': ['description', 'price', 'rating', 'rating_count', 'accomodation_type', 'property_amenities', 'room_amenities', 'wellness_spa', 'accessibility', 'for_children', 'city', 'country'], 'city': 'London', 'min_price': None, 'max_price': None, 'min_rating': None, 'max_rating': None, 'accomodation_types_include': 'Hotel', 'accomodation_types_exclude': None, 'property_amenities_include_all': ['Pets allowed'], 'property_amenities_include_any': None, 'property_amenities_exclude': None, 'room_amenities_include_all': None, 'room_amenities_include_any': None, 'room_amenities_exclude': None, 'wellness_spa_include_all': None, 'wellness_spa_include_any': None, 'wellness_spa_exclude': None, 'accessibility_include_all': None, 'accessibility_include_any': None, 'accessibility_exclude': None, 'for_children_include_all': Non

,description,price,rating,rating_count,accomodation_type,property_amenities,room_amenities,wellness_spa,accessibility,for_children,city,country,id,similarity_score
0,"Welcome to Canopy by Hilton London City, an im...",145.0,9.4,3119,Hotel,"[24-hour reception, 24-hour room service, Busi...","[Air conditioning, Bathtub, Cable TV, Central ...",[],"[Accessible bathroom, Accessible hotel, In-roo...",[Cot],London,United Kingdom,Canopy by Hilton London City,0.798498
1,Begin your day with a scrumptious on-site brea...,120.0,9.1,1867,Hotel,"[24-hour reception, Café, Cashless payment, Co...","[Air conditioning, Balcony, Central heating, F...",[],[Wheelchair accessible],[Cot],London,United Kingdom,"The Gantry London, Curio Collection By Hilton",0.793067
2,The Marylebone is centrally located in the hea...,327.0,8.9,7258,Hotel,"[24-hour reception, Breakfast, Business centre...","[Air conditioning, Bathtub, Cable TV, Central ...",[Sauna],"[Accessible hotel, Wheelchair accessible]",[Childcare],London,United Kingdom,The Marylebone Hotel,0.789340


In [6]:
rows = []

space_names = ["description", "price", "rating", "rating_count"]
# order of spaces is the same as during index creation

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)

df = pd.DataFrame(rows)
df

,id,description,price,rating,rating_count
0,Canopy by Hilton London City,0.300717,0.0,0.497781,0.0
1,"The Gantry London, Curio Collection By Hilton",0.298055,0.0,0.495012,0.0
2,The Marylebone Hotel,0.296786,0.0,0.492555,0.0
